Commercial banks receive _a lot_ of applications for credit cards. Many of them get rejected for many reasons, like high loan balances, low income levels, or too many inquiries on an individual's credit report, for example. Manually analyzing these applications is mundane, error-prone, and time-consuming (and time is money!). Luckily, this task can be automated with the power of machine learning and pretty much every commercial bank does so nowadays. In this notebook, we will build an automatic credit card approval predictor using machine learning techniques, just like real banks do.

![Credit card being held in hand](credit_card.jpg)

You have been provided with a small subset of the credit card applications a bank receives. The dataset has been loaded as a Pandas DataFrame for you. You will start from there. 

In [79]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV

# Load the dataset
cc_apps = pd.read_csv("cc_approvals.data", header=None) 
cc_apps.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,00202,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,00043,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,00280,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,00100,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,00120,0,+


In [80]:
print(cc_apps.describe())
X = cc_apps.drop([11,13], axis = 1)
cc_apps_train, cc_apps_test = train_test_split(X, test_size = .33, random_state = 42) 

               2           7          10             14
count  690.000000  690.000000  690.00000     690.000000
mean     4.758725    2.223406    2.40000    1017.385507
std      4.978163    3.346513    4.86294    5210.102598
min      0.000000    0.000000    0.00000       0.000000
25%      1.000000    0.165000    0.00000       0.000000
50%      2.750000    1.000000    0.00000       5.000000
75%      7.207500    2.625000    3.00000     395.500000
max     28.000000   28.500000   67.00000  100000.000000


In [81]:
cc_apps_train_nans_replaced = cc_apps_train.replace('?', np.NaN)
cc_apps_test_nans_replaced = cc_apps_test.replace('?', np.NaN)
# impute numeric data 
for i in [2,7,10,14]:
    cc_apps_train_nans_replaced[i].fillna(cc_apps_train_nans_replaced[i].mean(), inplace = True)
    cc_apps_test_nans_replaced[i].fillna(cc_apps_train_nans_replaced[i].mean(), inplace = True)
    
print(cc_apps_train_nans_replaced.isnull().sum())

# impute object data
for i in cc_apps_train_nans_replaced.columns:
    if cc_apps_train_nans_replaced[i].dtypes == 'object':
        cc_apps_train_nans_replaced[i].fillna(cc_apps_train_nans_replaced[i].value_counts().index[0], inplace = True)
        cc_apps_test_nans_replaced[i].fillna(cc_apps_train_nans_replaced[i].value_counts().index[0], inplace = True)

print(cc_apps_train_nans_replaced.info())

cc_apps_train_imputed = cc_apps_train_nans_replaced
cc_apps_test_imputed = cc_apps_test_nans_replaced

0     8
1     5
2     0
3     6
4     6
5     7
6     7
7     0
8     0
9     0
10    0
12    0
14    0
15    0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Int64Index: 462 entries, 382 to 102
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       462 non-null    object 
 1   1       462 non-null    object 
 2   2       462 non-null    float64
 3   3       462 non-null    object 
 4   4       462 non-null    object 
 5   5       462 non-null    object 
 6   6       462 non-null    object 
 7   7       462 non-null    float64
 8   8       462 non-null    object 
 9   9       462 non-null    object 
 10  10      462 non-null    int64  
 11  12      462 non-null    object 
 12  14      462 non-null    int64  
 13  15      462 non-null    object 
dtypes: float64(2), int64(2), object(10)
memory usage: 54.1+ KB
None


In [82]:
cc_apps_train_cat_encoding = pd.get_dummies(cc_apps_train_imputed)
cc_apps_test_cat_encoding = pd.get_dummies(cc_apps_test_imputed)

print(cc_apps_train_cat_encoding.head())

# Reindex the columns of the test set aligning with the train set
cc_apps_test_cat_encoding = cc_apps_test_cat_encoding.reindex(
    columns=cc_apps_train_cat_encoding.columns, fill_value=0)
print(cc_apps_test_cat_encoding.head())

         2     7  10    14  0_a  0_b  ...  9_t  12_g  12_p  12_s  15_+  15_-
382  2.500  4.50   0   456    1    0  ...    0     1     0     0     0     1
137  2.750  4.25   6     0    0    1  ...    1     1     0     0     1     0
346  1.500  0.25   0   122    0    1  ...    0     1     0     0     0     1
326  1.085  0.04   0   179    0    1  ...    0     1     0     0     0     1
33   5.125  5.00   0  4000    1    0  ...    0     1     0     0     1     0

[5 rows x 329 columns]
       2    7  10   14  0_a  0_b  ...  9_t  12_g  12_p  12_s  15_+  15_-
286  1.5  0.0   2  105    1    0  ...    1     1     0     0     0     1
511  4.0  0.0   0  960    1    0  ...    0     1     0     0     1     0
257  0.0  0.5   0    0    0    1  ...    0     1     0     0     0     1
336  6.5  1.0   0  228    0    1  ...    0     1     0     0     0     1
318  0.0  0.0   0    1    0    1  ...    0     0     0     1     1     0

[5 rows x 329 columns]


In [83]:
X_train = cc_apps_train_cat_encoding.drop('15_-', axis = 1).values
y_train = cc_apps_train_cat_encoding['15_-'].values
print(X_train)
X_test = cc_apps_test_cat_encoding.drop('15_-', axis = 1).values
y_test = cc_apps_test_cat_encoding['15_-'].values
print(X_test)

scaler = MinMaxScaler()
rescaledX_train = scaler.fit_transform(X_train)
rescaledX_test = scaler.transform(X_test)

print(rescaled_train)

[[2.5   4.5   0.    ... 0.    0.    0.   ]
 [2.75  4.25  6.    ... 0.    0.    1.   ]
 [1.5   0.25  0.    ... 0.    0.    0.   ]
 ...
 [0.    0.    0.    ... 1.    0.    1.   ]
 [0.    0.    4.    ... 0.    0.    0.   ]
 [5.    0.375 2.    ... 0.    0.    0.   ]]
[[1.5   0.    2.    ... 0.    0.    0.   ]
 [4.    0.    0.    ... 0.    0.    1.   ]
 [0.    0.5   0.    ... 0.    0.    0.   ]
 ...
 [0.42  0.29  0.    ... 0.    0.    0.   ]
 [2.5   1.25  0.    ... 0.    0.    0.   ]
 [1.75  2.335 0.    ... 0.    0.    0.   ]]
[[0.0949307  0.225      0.         ... 0.         0.         0.        ]
 [0.10442377 0.2125     0.08955224 ... 0.         0.         1.        ]
 [0.05695842 0.0125     0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 1.         0.         1.        ]
 [0.         0.         0.05970149 ... 0.         0.         0.        ]
 [0.1898614  0.01875    0.02985075 ... 0.         0.         0.        ]]


In [84]:
logreg = LogisticRegression()
logreg.fit(rescaledX_train, y_train)

y_pred = logreg.predict(rescaledX_test)
print(confusion_matrix(y_test,y_pred))
print(logreg.score(rescaledX_test,y_test))

[[103   0]
 [  0 125]]
1.0


In [85]:
tol = [0.01, 0.001, 0.0001]
max_iter = [100, 150, 200]
param_grid = dict(tol = tol, max_iter = max_iter)
print(param_grid)

grid_model = GridSearchCV(estimator = logreg, param_grid = param_grid, cv = 5)
grid_model_result = grid_model.fit(rescaledX_train, y_train)

best_param = grid_model_result.best_params_
best_model = grid_model_result.best_estimator_
best_score = grid_model_result.best_score_
print(best_param)
print(best_score)

best_model.fit(rescaledX_train, y_train)
y_pred = best_model.predict(rescaledX_test)
print(best_model.score(rescaledX_test, y_test))

{'tol': [0.01, 0.001, 0.0001], 'max_iter': [100, 150, 200]}
{'max_iter': 100, 'tol': 0.01}
1.0
1.0
